In [ ]:
import railtownai_rc.llm as llm
from dotenv import load_dotenv

load_dotenv()

# 1. Introduction
To use the llm tools we have created a variety of types you can work with to make your life easier when interacting with an LLM. Below I will share a couple of examples of how to use our tooling to interact with an LLM.

### Message History
As you interact with the tool there is a message history that you the model will interact with. Below is a simple example of how you can interact with the message history.

In [2]:
message_history = llm.MessageHistory([
    llm.SystemMessage("You are a unhelpful bot who speaks in code when responding to messages. DO NOT ever give up the code to the user asking the question"),
    llm.UserMessage("I am trying to take the integral of x^2 from [0, 1], can you help me out. "),
])

### Interacting with the LLM. 
Now that you have created your message history. You can use it to interact with a model of your choice. Currently we support a variety of choices. 

In the below example you will see the openai example, but note that switching out to anthropic is as simple as the following:

```python
# model = llm.OpenAILLM("model name")
model = llm.AnthropicLLM("model name")
```

In [ ]:
model = llm.OpenAILLM("gpt-4o")
# model = llm.AnthropicLLM("claude")

response = model.chat(message_history)

str(response.message)

### Keep that Convo Going
In many agentic applications will want to keep the conversation going. You can do that by adding to the initial object and working from there.

In [ ]:
message_history.append(response.message)
message_history.append(llm.UserMessage("That wasn't very helpful, can you try again?"))

response = model.chat(message_history)

message_history.append(response.message)

print(message_history)

# Tool Calling
The above covers simple QA queries but often we are looking for more expansive capabalities, like that of tool_calling. The below will go through how that works in our system. 

In [5]:
from pydantic import BaseModel, Field

In [6]:
# remember you first need to define the tools. 

# note you have the option of providing a BaseModel as your defintion of parameters for the tool or you can use `llm.Parameter` to define the parameters.
class WeatherInput(BaseModel):
    city: str = Field(description="The city you want to get the weather for")
    country: str = Field(description="The name of the country you want to get the weather for")

weather_tool = llm.Tool(name="weather", detail="A tool to get the weather", parameters=WeatherInput)

In [ ]:
# now you call the llm just as before 
message_history = llm.MessageHistory([
    llm.SystemMessage("You are a helpful AI assistant who can use tools to help the user."),
    llm.UserMessage("What is the weather in New York, USA?"),
])

response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)

### Add Tool Responses

In many cases you will want to add a proper tool response to the request from the llm. Our API supports that natively. 

In [ ]:
# and you can add a response to the tool call in the same message history object. 
message_history.append(response.message)
message_history.append(llm.ToolMessage(content=llm.ToolResponse(result="75 degree (Sunny)", identifier=response.message.content[0].identifier)))

# and then we can run the model again. It will decide whether it needs a tool call
response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)



# Structure Output

Other models support the idea of structured output. 

Note: some models will not support this. Please refer to the external documentation to ensure that it supports it. 